In [1]:
from pynse import *
import os
import datetime
import matplotlib.pyplot as plt
#import mplfinance as mpf
import time, asyncio
import matplotlib.dates as mdates
import mplfinance as mpf
import statsmodels.api as sm
import numpy as np
from datetime import datetime


import os
import glob
import pandas as pd
import mplfinance as mpf

nse=Nse()
#nse.bhavcopy(series="all")
#bhavcopy_full=nse.bhavcopy(series="all")

MANAGING FOLDER

In [2]:
import os
import shutil
import glob
import datetime

source_folder = "C:/Users/99111/OneDrive/Documents/pynse/Shashank work/data/raw/Index data"
destination_folder = "C:/Users/99111/OneDrive/Documents/pynse/Shashank work/data/raw/Index data full"

target_date = datetime.datetime(2024, 1, 8)  # Change this to your desired date

# Get a list of all files in the source folder
file_list = glob.glob(os.path.join(source_folder, '*'))

# Move the files to the destination folder based on the target date
files_moved = 0
for file_path in file_list:
    file_name = os.path.basename(file_path)
    modified_time = datetime.datetime.fromtimestamp(os.path.getmtime(file_path))
    
    if modified_time <= target_date:
        destination_path = os.path.join(destination_folder, file_name)
        shutil.move(file_path, destination_path)
        files_moved += 1

print(f"{files_moved} files moved to the destination folder.")



0 files moved to the destination folder.


MAKING CHORONOLOGY CHARTS

In [ ]:
from pynse import *
import os
import datetime
import matplotlib.pyplot as plt
#import mplfinance as mpf
import time, asyncio
import matplotlib.dates as mdates
import mplfinance as mpf
import statsmodels.api as sm
import numpy as np
from datetime import datetime


import os
import glob
import pandas as pd
import mplfinance as mpf

nse=Nse()
import os
import pandas as pd
import glob
from datetime import timezone, timedelta

# Read first set of CSV files
folder_path1 = "C:/Users/99111/OneDrive/Documents/pynse/Shashank work/data/raw/Index data/*.csv"

files1 = glob.glob(folder_path1)
df_list1 = []
for file in files1:
    # Get file name and date of creation
    file_name = os.path.basename(file)
    created_date_utc = pd.to_datetime(os.stat(file).st_mtime, unit='s', utc=True)
    created_date_local = created_date_utc.replace(tzinfo=timezone.utc).astimezone(tz=timezone(timedelta(hours=5, minutes=30)))
    # Read CSV file
    df = pd.read_csv(file)
    # Add new columns
    df['File_Name'] = file_name
    df['File_Created_Date'] = created_date_local.strftime('%Y-%m-%d %H:%M')
    df['File_Date'] = created_date_local.strftime('%Y-%m-%d')
    # Append to list
    df_list1.append(df)
# Concatenate all dataframes
data = pd.concat(df_list1, ignore_index=True)
data['File_Date'] = pd.to_datetime(data['File_Date'])
data = data.fillna(0)

data = data[data['File_Date'] >= pd.to_datetime('2023-10-25')]


expiries = data['expiryDate'].unique().tolist()
date_series = pd.Series(pd.to_datetime(expiries))
today = pd.to_datetime(datetime.today().date())
future_dates = date_series[date_series >= today]
sorted_dates = future_dates.sort_values()
next_2_closest_dates = sorted_dates.head(2)
Next2Expiries = next_2_closest_dates.dt.strftime('%Y-%m-%d').tolist()

result_df = pd.DataFrame()

symbols = ['NIFTY','BANKNIFTY']

for symbol in symbols:
            try:
                d2 = data[(data['PE.underlying'] == symbol) | (data['CE.underlying'] == symbol)]
                
                expiries = d2['expiryDate'].unique().tolist()
                date_series = pd.Series(pd.to_datetime(expiries))
                today = pd.to_datetime(datetime.today().date())
                future_dates = date_series[date_series >= today]
                sorted_dates = future_dates.sort_values()
                next_2_closest_dates = sorted_dates.head(2)
                Next2Expiries = next_2_closest_dates.dt.strftime('%Y-%m-%d').tolist()

                for Next2Expiry in Next2Expiries:
                    try:
                                d1 = d2[d2['expiryDate'] == Next2Expiry]
                                filenames = d1['File_Name'].unique().tolist()
                                
                                for filename in filenames:
                                    try:
                                            d = d1[d1['File_Name'] == filename]
                                            d = d.sort_values(by='strikePrice', ascending=True)
                                            d.loc[:, 'diff'] = d['strikePrice'] - d['strikePrice'].shift(1)
                                            diff = d.loc[d['diff'] != 0, 'diff'].median()
                                            cpcol = d['CE.underlyingValue'][(d['CE.underlyingValue'] != '') & (d['CE.underlyingValue'] != 0)]
                                            cp = cpcol.median()
                                       
                                            dt = d['File_Created_Date'].tolist()[0]
                                            expdt = d['expiryDate'].tolist()[0]

                                            # Price And Return %
                                            d.loc[:, 'callprice'] = (d['CE.bidQty']*d['CE.bidprice'] + d['CE.askQty']*d['CE.askPrice']) / (d['CE.bidQty'] + d['CE.askQty'])
                                            d.loc[:, 'putprice'] = (d['PE.bidQty']*d['PE.bidprice'] + d['PE.askQty']*d['PE.askPrice']) / (d['PE.bidQty'] + d['PE.askQty'])
                                            d.loc[:, 'callreturn'] = (d['callprice'] / cp) * 100
                                            d.loc[:, 'putreturn'] = (d['putprice'] / cp) * 100
                                            d['calloversold'] = (d['CE.totalSellQuantity'] - d['CE.totalBuyQuantity']).clip(lower=0)
                                            d['putoversold'] = (d['PE.totalSellQuantity'] - d['PE.totalBuyQuantity']).clip(lower=0)
                                            d = d.assign(sp_minus_cp = abs(d['strikePrice'] - cp))
                                            d = d.reset_index(drop=False)
                                            
                                            #workings to get 2 separate & comparable call DF and put DF
                                            ATM_SP_IDX = d['sp_minus_cp'].idxmin()
                                            ATM_SP = d.loc[ATM_SP_IDX, 'strikePrice']


                                            SPabove = ATM_SP_IDX
                                            SPbelow = d.shape[0] - ATM_SP_IDX - 1

                                            LongRowstopick = min(SPabove,SPbelow,30)
                                            ShortRowstopick = int(np.floor(LongRowstopick / 3))

                                            #At the DF row

                                            Atthemoneydf  = d.iloc[ATM_SP_IDX, :]


                                            #Subsets of Return 
                                            crdf = d[ATM_SP_IDX:ATM_SP_IDX+LongRowstopick+1].copy()
                                            crdf = crdf[['strikePrice', 'callreturn']]
                                            crdf = pd.pivot_table(crdf, columns='strikePrice', values='callreturn', aggfunc=lambda x: x.iloc[0])
                                            crdf = crdf.reset_index()
                                            
                                            prdf = d[ATM_SP_IDX-LongRowstopick:ATM_SP_IDX+1].copy()
                                            prdf = prdf[['strikePrice', 'putreturn']]
                                            prdf = pd.pivot_table(prdf, columns='strikePrice', values='putreturn', aggfunc=lambda x: x.iloc[0])
                                            prdf = prdf.reset_index()

                                            merged_df = pd.concat([prdf, crdf], axis=1)
                                            merged_df.columns = ['Column ' + str(i) for i in range(0, merged_df.shape[1])]

                                            prdf1 = prdf.iloc[:, prdf.shape[1]-1][0]
                                            prdf2 = prdf.iloc[:, prdf.shape[1]-2][0]
                                            prdf3 = prdf.iloc[:, prdf.shape[1]-3][0]
                                            prdf4 = prdf.iloc[:, prdf.shape[1]-4][0]
                                            prdf5 = prdf.iloc[:, prdf.shape[1]-5][0]
                                                
                                            crdf1 = crdf.iloc[:, 1][0]
                                            crdf2 = crdf.iloc[:, 2][0]
                                            crdf3 = crdf.iloc[:, 3][0]
                                            crdf4 = crdf.iloc[:, 4][0]
                                            crdf5 = crdf.iloc[:, 5][0]

                                            p1 = round((crdf1 - prdf1)/(max(prdf1,crdf1)), 3)*100
                                            p2 = round((crdf2 - prdf2)/(max(prdf2,crdf2)), 3)*100
                                            p3 = round((crdf3 - prdf3)/(max(prdf3,crdf3)), 3)*100
                                            p4 = round((crdf4 - prdf4)/(max(prdf4,crdf4)), 3)*100
                                            p5 = round((crdf5 - prdf5)/(max(prdf5,crdf5)), 3)*100
                                            #
                                            
                                            #
                                            #Calcing Min Lot Size
                                            minlot1 = d['CE.askQty'].unique().tolist()
                                            minlot2 = d['CE.askQty'].unique().tolist()
                                            minlot3 = d['PE.askQty'].unique().tolist()
                                            minlot4 = d['PE.askQty'].unique().tolist()
                                            cmin = list(filter(lambda x: x != 0, minlot1 + minlot2 + minlot3 + minlot4))
                                            cmindf = pd.DataFrame({'lots': cmin})
                                            minLotSize = cmindf['lots'].min()
                                            #Min lot size
                                            
                                            #Calcing Support and resistance
                                            calld = d[(d['strikePrice'] >= ATM_SP - diff*10) & (d['strikePrice'] <= ATM_SP + diff*30)].copy()

                                            TCbought = calld['CE.totalBuyQuantity'].sum()
                                            TCsold = calld['CE.totalSellQuantity'].sum()
                                            calld.loc[:, 'Weights1'] = calld['CE.totalSellQuantity'] / TCsold
                                            calld.loc[:, 'resistance1'] = calld['strikePrice'] * calld['Weights1']
                                            Resistance1 = calld['resistance1'].sum()

                                            TotalCallOS = calld['calloversold'].sum()
                                            if TotalCallOS == 0:
                                                TotalCallOS = 1
                                            calld.loc[:, 'Weights2'] = calld['calloversold'] / TotalCallOS
                                            calld.loc[:, 'resistance2'] = calld['strikePrice'] * calld['Weights2']
                                            Resistance2 = calld['resistance2'].sum()
                                            if Resistance2 <= 0:
                                                Resistance2 = Resistance1 
                                            Resistance = (Resistance1 + Resistance2)/2
                                                            

                                            putd = d[(d['strikePrice'] >= ATM_SP - diff*30) & (d['strikePrice'] <= ATM_SP + diff*10)].copy()
                                                            
                                            TPbought = putd['PE.totalBuyQuantity'].sum()
                                            TPsold = putd['PE.totalSellQuantity'].sum()
                                            putd.loc[:, 'Weights1'] = putd['PE.totalSellQuantity'] / TPsold
                                            putd.loc[:, 'Support1'] = putd['strikePrice'] * putd['Weights1']
                                            Support1 = putd['Support1'].sum()

                                            TotalPutOS = putd['putoversold'].sum()
                                            if TotalPutOS == 0:
                                                TotalPutOS = 1
                                            putd.loc[:, 'Weights2'] = putd['putoversold'] / TotalPutOS
                                            putd.loc[:, 'Support2'] = putd['strikePrice'] * putd['Weights2']
                                            Support2 = putd['Support2'].sum()
                                            if Support2 <= 0:
                                                Support2 = Support1        
                                            Support = (Support1 + Support2)/2
                                            #Calcing Support and resistance

                                            dataz = {
                                                "filename": [filename],
                                                "stockname": [symbol],
                                                "dt": [dt],
                                                "cp": [cp],
                                                "ATM_SP": [ATM_SP],
                                                "diff": [diff],

                                                "Acceptablepremdistortion": [((cp-ATM_SP)/diff)*100],

                                                "Support": [Support],
                                                "Resistance": [Resistance],
                                                "CallOversold": [TotalCallOS],
                                                "PutOversold": [TotalPutOS],
                                                "OS-Call/Put": [TotalCallOS/TotalPutOS],
                                                "OS-Put/Call": [TotalPutOS/TotalCallOS],
                                                "Call/Put": [TCsold/TPsold],
                                                "Put/Call": [TPsold/TCsold],
                                                "Expdt": [expdt],
                                                "P1": [p1],
                                                "P2": [p2],
                                                "P3": [p3],
                                                "P4": [p4],
                                                "P5": [p5],
                                                "PutSQ - CallSQ": [TotalPutOS-TotalCallOS],
                                                "Lotsize": [minLotSize],
                                                "percontractvalue": [minLotSize*cp],

                                                "Callsoldquantity": [TCsold],
                                                "PutsoldQuantity": [TPsold],
                                                "CallBuyquantity": [TCbought],
                                                "PutBuyQuantity": [TPbought],
                                            }

                                            dd = pd.DataFrame(dataz)

                                            fdf = pd.concat([dd, merged_df], axis=1)
                                            result_df = pd.concat([result_df, fdf], axis=0, ignore_index=True)

                                    except Exception as e:
                                        print(f"Error occurred while creating chart for {filename}: {str(e)}")
                                        continue

                    except Exception as e:
                        print(f"Error occurred while creating chart for {filename}: {str(e)}")
                        continue   
            except Exception as e:
                    print(f"Error occurred while creating chart for {symbol}: {str(e)}")
                    continue                        

#checking for only 2 decimal points
result_df[['Support', 'Resistance','OS-Call/Put','OS-Put/Call','Call/Put','Put/Call','Acceptablepremdistortion']] = result_df[['Support', 'Resistance','OS-Call/Put','OS-Put/Call','Call/Put','Put/Call','Acceptablepremdistortion']].round(2)

#sorting Dates
result_df['dt'] = pd.to_datetime(result_df['dt'])
result_df['Date'] = result_df['dt'].dt.date

file_path = r'C:\Users\99111\OneDrive\Desktop\daily updates\Latest Charts\INDEX\Excel Data For Whole Picture\Latest_INDEX_Option_market_updates_test.xlsx'
result_df.to_excel(file_path, index=False)


# MAKING CHARTS #

import pandas as pd
import os
import glob

folder_path1 = "C:/Users/99111/OneDrive/Desktop/daily updates/Latest Charts/INDEX/Excel Data For Whole Picture/*.xlsx"


files1 = glob.glob(folder_path1)
df_list1 = []

for file in files1:
    df = pd.read_excel(file)
    df_list1.append(df)

# Concatenate all data frames into a single data frame
df = pd.concat(df_list1, ignore_index=True)
df = df.sort_values('dt')

symbols = ['NIFTY', 'BANKNIFTY']

dff = df

save_dir = r'C:\Users\99111\OneDrive\Desktop\daily updates\Latest Charts\INDEX\whole Pictures'

for symbol in symbols:
    try:
        
        df1 = dff[dff['stockname'] == symbol]
        expiries = df1['Expdt'].unique().tolist()
        
        for expiry in expiries:
            try:
                df2 = df1[df1['Expdt'] == expiry]

                fig, axs = plt.subplots(2, 1, figsize=(20, 10))

                # Create the first subplot for columns A and B
                x = df2['dt']  # Assuming 'Date' is the column you want to use for x-axis
                axs[0].plot(x, df2['Support'], label='support')
                axs[0].plot(x, df2['cp'], label='cp')
                axs[0].plot(x, df2['Resistance'], label='Resistance')
                axs[0].axhline(df2['cp'].max(), color='black', linestyle='--')
                axs[0].axhline(df2['cp'].min(), color='black', linestyle='--')
                axs[0].axhline(df2['cp'].iloc[-1], color='red', linestyle='--')
                
                axs[0].legend()
                axs[0].set_title(symbol)

                # Set y-axis limits based on maximum resistance and minimum support
                y_min = df2['Support'].min()
                y_max = df2['Resistance'].max()
                axs[0].set_ylim(y_min, y_max)

                ax2 = axs[1].twinx()
                axs[1].plot(x, df2['Call/Put'], label='Call/Put' ,color=(1.0, 0.5, 0.5), linestyle='--')
                axs[1].plot(x, df2['Put/Call'], label='Put/Call', color=(0.7, 0.9, 0.7), linestyle='--')
                axs[1].axhline(1.3, color='black')

                ax2.plot(x, df2['Callsoldquantity'], label='Callsoldquantity', color = 'red')
                ax2.plot(x, df2['PutsoldQuantity'], label='PutsoldQuantity',color = 'green' )

                # Set labels and titles for both y-axes
                #axs[1].set_ylabel('Call TSQ / Put TSQ')
                #ax2.set_ylabel('OversoldQuantity')

                # Combine the legends from both axes
                lines, labels = axs[1].get_legend_handles_labels()
                lines2, labels2 = ax2.get_legend_handles_labels()
                axs[1].legend(lines + lines2, labels + labels2)

                axs[1].set_title(expiry)

                # Adjust the spacing to avoid overlapping labels
                fig.tight_layout()

                chart_file = os.path.join(save_dir, f"{symbol}_{expiry}.jpg")
                plt.savefig(chart_file, dpi=100)

                # Close the figure to release resources
                plt.close(fig)

            except Exception as e:
                print(f"Error occurred while an expiry date for {expiry}: {str(e)}")
                continue

    except Exception as e:
        print(f"Error occurred while an expiry date for {symbol}: {str(e)}")
        continue

With price analysis

In [2]:
from pynse import *
import os
import datetime
import matplotlib.pyplot as plt
#import mplfinance as mpf
import time, asyncio
import matplotlib.dates as mdates
import mplfinance as mpf
import statsmodels.api as sm
import numpy as np
from datetime import datetime

import os
import glob
import pandas as pd
import mplfinance as mpf

nse=Nse()
import os
import pandas as pd
import glob
from datetime import timezone, timedelta

# Read first set of CSV files
folder_path1 = "C:/Users/99111/OneDrive/Documents/pynse/Shashank work/data/raw/Index data/*.csv"

files1 = glob.glob(folder_path1)
df_list1 = []
for file in files1:
    # Get file name and date of creation
    file_name = os.path.basename(file)
    created_date_utc = pd.to_datetime(os.stat(file).st_mtime, unit='s', utc=True)
    created_date_local = created_date_utc.replace(tzinfo=timezone.utc).astimezone(tz=timezone(timedelta(hours=5, minutes=30)))
    # Read CSV file
    df = pd.read_csv(file)
    # Add new columns
    df['File_Name'] = file_name
    df['File_Created_Date'] = created_date_local.strftime('%Y-%m-%d %H:%M')
    df['File_Date'] = created_date_local.strftime('%Y-%m-%d')
    # Append to list
    df_list1.append(df)
# Concatenate all dataframes
data = pd.concat(df_list1, ignore_index=True)
data['File_Date'] = pd.to_datetime(data['File_Date'])
data = data.fillna(0)

data = data[data['File_Date'] >= pd.to_datetime('2023-10-25')]


expiries = data['expiryDate'].unique().tolist()
date_series = pd.Series(pd.to_datetime(expiries))
today = pd.to_datetime(datetime.today().date())
future_dates = date_series[date_series >= today]
sorted_dates = future_dates.sort_values()
next_2_closest_dates = sorted_dates.head(2)
Next2Expiries = next_2_closest_dates.dt.strftime('%Y-%m-%d').tolist()

result_df = pd.DataFrame()
newresults_df = pd.DataFrame()

symbols = ['NIFTY','BANKNIFTY']

for symbol in symbols:
            try:
                d2 = data[(data['PE.underlying'] == symbol) | (data['CE.underlying'] == symbol)]
                
                expiries = d2['expiryDate'].unique().tolist()
                date_series = pd.Series(pd.to_datetime(expiries))
                today = pd.to_datetime(datetime.today().date())
                future_dates = date_series[date_series >= today]
                sorted_dates = future_dates.sort_values()
                next_2_closest_dates = sorted_dates.head(2)
                Next2Expiries = next_2_closest_dates.dt.strftime('%Y-%m-%d').tolist()

                for Next2Expiry in Next2Expiries:
                    try:
                                d1 = d2[d2['expiryDate'] == Next2Expiry]
                                filenames = d1['File_Name'].unique().tolist()
                                
                                for filename in filenames:
                                    try:
                                            d = d1[d1['File_Name'] == filename]
                                            d = d.sort_values(by='strikePrice', ascending=True)
                                            d.loc[:, 'diff'] = d['strikePrice'] - d['strikePrice'].shift(1)
                                            diff = d.loc[d['diff'] != 0, 'diff'].median()
                                            cpcol = d['CE.underlyingValue'][(d['CE.underlyingValue'] != '') & (d['CE.underlyingValue'] != 0)]
                                            cp = cpcol.median()
                                       
                                            dt = d['File_Created_Date'].tolist()[0]
                                            expdt = d['expiryDate'].tolist()[0]

                                            # Price And Return %
                                            d.loc[:, 'callprice'] = (d['CE.bidQty']*d['CE.bidprice'] + d['CE.askQty']*d['CE.askPrice']) / (d['CE.bidQty'] + d['CE.askQty'])
                                            d.loc[:, 'putprice'] = (d['PE.bidQty']*d['PE.bidprice'] + d['PE.askQty']*d['PE.askPrice']) / (d['PE.bidQty'] + d['PE.askQty'])
                                            d.loc[:, 'callreturn'] = (d['callprice'] / cp) * 100
                                            d.loc[:, 'putreturn'] = (d['putprice'] / cp) * 100
                                            d['calloversold'] = (d['CE.totalSellQuantity'] - d['CE.totalBuyQuantity']).clip(lower=0)
                                            d['putoversold'] = (d['PE.totalSellQuantity'] - d['PE.totalBuyQuantity']).clip(lower=0)
                                            d = d.assign(sp_minus_cp = abs(d['strikePrice'] - cp))
                                            d = d.reset_index(drop=False)
                                            
                                            #workings to get 2 separate & comparable call DF and put DF
                                            ATM_SP_IDX = d['sp_minus_cp'].idxmin()
                                            ATM_SP = d.loc[ATM_SP_IDX, 'strikePrice']


                                            SPabove = ATM_SP_IDX
                                            SPbelow = d.shape[0] - ATM_SP_IDX - 1

                                            LongRowstopick = min(SPabove,SPbelow,30)
                                            ShortRowstopick = int(np.floor(LongRowstopick / 3))

                                            #At the DF row

                                            Atthemoneydf  = d.iloc[ATM_SP_IDX, :]

                                            #Calcing Min Lot Size
                                            minlot1 = d['CE.askQty'].unique().tolist()
                                            minlot2 = d['CE.askQty'].unique().tolist()
                                            minlot3 = d['PE.askQty'].unique().tolist()
                                            minlot4 = d['PE.askQty'].unique().tolist()
                                            cmin = list(filter(lambda x: x != 0, minlot1 + minlot2 + minlot3 + minlot4))
                                            cmindf = pd.DataFrame({'lots': cmin})
                                            minLotSize = cmindf['lots'].min()
                                            #Min lot size
                                            
                                            #Calcing Support and resistance
                                            calld = d[(d['strikePrice'] >= ATM_SP - diff*10) & (d['strikePrice'] <= ATM_SP + diff*30)].copy()

                                            TCbought = calld['CE.totalBuyQuantity'].sum()
                                            TCsold = calld['CE.totalSellQuantity'].sum()
                                            calld.loc[:, 'Weights1'] = calld['CE.totalSellQuantity'] / TCsold
                                            calld.loc[:, 'resistance1'] = calld['strikePrice'] * calld['Weights1']
                                            Resistance1 = calld['resistance1'].sum()

                                            TotalCallOS = calld['calloversold'].sum()
                                            if TotalCallOS == 0:
                                                TotalCallOS = 1
                                            calld.loc[:, 'Weights2'] = calld['calloversold'] / TotalCallOS
                                            calld.loc[:, 'resistance2'] = calld['strikePrice'] * calld['Weights2']
                                            Resistance2 = calld['resistance2'].sum()
                                            if Resistance2 <= 0:
                                                Resistance2 = Resistance1 
                                            Resistance = (Resistance1 + Resistance2)/2
                                                            

                                            putd = d[(d['strikePrice'] >= ATM_SP - diff*30) & (d['strikePrice'] <= ATM_SP + diff*10)].copy()
                                                            
                                            TPbought = putd['PE.totalBuyQuantity'].sum()
                                            TPsold = putd['PE.totalSellQuantity'].sum()
                                            putd.loc[:, 'Weights1'] = putd['PE.totalSellQuantity'] / TPsold
                                            putd.loc[:, 'Support1'] = putd['strikePrice'] * putd['Weights1']
                                            Support1 = putd['Support1'].sum()

                                            TotalPutOS = putd['putoversold'].sum()
                                            if TotalPutOS == 0:
                                                TotalPutOS = 1
                                            putd.loc[:, 'Weights2'] = putd['putoversold'] / TotalPutOS
                                            putd.loc[:, 'Support2'] = putd['strikePrice'] * putd['Weights2']
                                            Support2 = putd['Support2'].sum()
                                            if Support2 <= 0:
                                                Support2 = Support1        
                                            Support = (Support1 + Support2)/2
                                            #Calcing Support and resistance

                                            dataz = {
                                                "filename": [filename],
                                                "stockname": [symbol],
                                                "dt": [dt],
                                                "cp": [cp],
                                                "ATM_SP": [ATM_SP],
                                                "diff": [diff],

                                                "Acceptablepremdistortion": [((cp-ATM_SP)/diff)*100],

                                                "Support": [Support],
                                                "Resistance": [Resistance],
                                                "CallOversold": [TotalCallOS],
                                                "PutOversold": [TotalPutOS],
                                                "OS-Call/Put": [TotalCallOS/TotalPutOS],
                                                "OS-Put/Call": [TotalPutOS/TotalCallOS],
                                                "Call/Put": [TCsold/TPsold],
                                                "Put/Call": [TPsold/TCsold],
                                                "Expdt": [expdt],
                                                "PutSQ - CallSQ": [TotalPutOS-TotalCallOS],
                                                "Lotsize": [minLotSize],
                                                "percontractvalue": [minLotSize*cp],

                                                "Callsoldquantity": [TCsold],
                                                "PutsoldQuantity": [TPsold],
                                                "CallBuyquantity": [TCbought],
                                                "PutBuyQuantity": [TPbought],
                                            }

                                            dd = pd.DataFrame(dataz)
                                            result_df = pd.concat([result_df, dd], axis=0, ignore_index=True)

####################################################################################################################################################
                                            

                                            #################Price Analysis###############3

                                            desired_rows = 10

                                            crdf = d[(d['strikePrice'] >= ATM_SP) & (d['strikePrice'] <= ATM_SP + diff*15)].copy()
                                            crdf = crdf[['strikePrice', 'callreturn','callprice']]
                                            crdf = crdf.reset_index()

                                            current_rows = len(crdf)

                                            # Add rows with zeros to the top if needed
                                            if current_rows < desired_rows:
                                                zero_rows = pd.DataFrame([[0] * len(crdf.columns)] * (desired_rows - current_rows), columns=crdf.columns)
                                                crdf = pd.concat([crdf , zero_rows], ignore_index=True)

                                            # Select the last 10 rows
                                            crdf = crdf.head(desired_rows)
                                            crdf = crdf.reset_index(drop=True)
                                            crdf['diff'] = crdf['strikePrice'] - crdf['strikePrice'].iloc[0]

                                            ##

                                            prdf = d[(d['strikePrice'] >= ATM_SP - diff*15) & (d['strikePrice'] <= ATM_SP)].copy()
                                            prdf = prdf[['strikePrice', 'putreturn', 'putprice']]
                                            prdf = prdf.reset_index()


                                            current_rows = len(prdf)

                                            # Add rows with zeros to the top if needed
                                            if current_rows < desired_rows:
                                                zero_rows = pd.DataFrame([[0] * len(prdf.columns)] * (desired_rows - current_rows), columns=prdf.columns)
                                                prdf = pd.concat([zero_rows , prdf], ignore_index=True)

                                            # Select the last 10 rows
                                            prdf = prdf.tail(desired_rows)
                                            prdf = prdf.reset_index(drop=True)
                                            prdf['diff'] = prdf['strikePrice'].iloc[-1] - prdf['strikePrice']


                                            ############FINAL OUTPUT##################
                                            merged_df = pd.merge(crdf, prdf, on='diff' , how='outer')
                                            merged_df = merged_df.rename(columns={'strikePrice_x': 'Call_SP', 'strikePrice_y': 'Put_SP'})
                                            merged_df = merged_df.drop(['index_x', 'index_y'], axis=1)

                                            merged_df['stockname'] = symbol
                                            merged_df['dt'] = dt
                                            merged_df['cp'] = cp
                                            merged_df['expdt'] = expdt

                                            merged_df = merged_df.sort_values('diff', ascending=True)
                                            merged_df = merged_df[merged_df['diff'] >= 0]
                                            merged_df = merged_df.reset_index(drop=True)
                                            merged_df['cpoints'] = merged_df['Call_SP'] - merged_df['cp']
                                            merged_df['ppoints'] = merged_df['cp'] - merged_df['Put_SP']
                                            merged_df['cpp'] = merged_df['callprice'] / merged_df['cpoints'] * 100
                                            merged_df['ppp'] = merged_df['putprice'] / merged_df['ppoints'] * 100

                                            merged_df.loc[0, 'cpp'] = (merged_df['callprice'].iloc[0] / merged_df['callprice'].iloc[1]) * merged_df['cpp'].iloc[1]
                                            merged_df.loc[0, 'ppp'] = (merged_df['putprice'].iloc[0] / merged_df['putprice'].iloc[1]) * merged_df['ppp'].iloc[1]
                                            merged_df[['cpp', 'ppp', 'callprice', 'putprice']] = merged_df[['cpp', 'ppp', 'callprice', 'putprice']].round(2)

                                            merged_df = merged_df[['stockname', 'dt', 'expdt','cp', 'Call_SP', 'cpp', 'diff', 'ppp', 'Put_SP','callprice','putprice']]
                                            merged_df['filename'] = filename

                                            newresults_df = pd.concat([newresults_df, merged_df], axis=0, ignore_index=True)

#################################################################################################################################333


                                    except Exception as e:
                                        print(f"Error occurred while creating chart for {filename}: {str(e)}")
                                        continue

                    except Exception as e:
                        print(f"Error occurred while creating chart for {filename}: {str(e)}")
                        continue   
            except Exception as e:
                    print(f"Error occurred while creating chart for {symbol}: {str(e)}")
                    continue                        

#checking for only 2 decimal points
result_df[['Support', 'Resistance','OS-Call/Put','OS-Put/Call','Call/Put','Put/Call','Acceptablepremdistortion']] = result_df[['Support', 'Resistance','OS-Call/Put','OS-Put/Call','Call/Put','Put/Call','Acceptablepremdistortion']].round(2)

#sorting Dates
result_df['dt'] = pd.to_datetime(result_df['dt'])
result_df['Date'] = result_df['dt'].dt.date

file_path = r'C:\Users\99111\OneDrive\Desktop\daily updates\Latest Charts\INDEX\Excel Data For Whole Picture\Latest_INDEX_Option_market_updates_test.xlsx'
result_df.to_excel(file_path, index=False)

directory_1 = r'C:\Users\99111\OneDrive\Desktop\daily updates\Latest Charts\INDEX'
file_name_1 = f'AllProcessedData_3.xlsx'
file_path_1 = os.path.join(directory_1, file_name_1)
newresults_df.to_excel(file_path_1, index=False)



